In [1]:
import os
import numpy as np
from PIL import Image
import math
import shutil

In [11]:
raw2 = os.listdir('raw2')
n_raw = 100

for i in range(33):
    os.rename('raw/' + str(i+201) + '.jpg', 'raw/' + str(i+n_raw) + '.jpg')
    os.rename('raw/' + str(i+201) + '.txt', 'raw/' + str(i+n_raw) + '.txt')

In [12]:
def rotate(origin, point, angle):
    ox, oy = origin
    px, py = point
    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy
# Funktion die Punkte rotiert

def augment(axis, mode, numrot, folder):
    os.makedirs(folder, exist_ok=True)
    try:
        shutil.copyfile('raw/classes.txt', folder + '/classes.txt')
    except:
        pass

    for i in range(133):
        writearray=[]
        image=Image.open("raw/"+str(i)+".jpg")
        image=np.array(image)

        if mode==1:
            if axis==0:
                image=np.flip(image, axis=0)

            if axis==1:
                image=np.flip(image, axis=1)

            else:
                image=np.flip(image, axis=0)
                image=np.flip(image, axis=1)

        if mode==0:
            if numrot==1:
                image=np.rot90(image)

            if numrot==2:
                image=np.rot90(image)
                image=np.rot90(image)

            if numrot==3:
                image=np.rot90(image)
                image=np.rot90(image)
                image=np.rot90(image)
            
        image=Image.fromarray(image)
        image.save(folder+"/"+str(i)+".jpg")
        # looped durch bilder und flipped/rotiert

        with open("raw/"+str(i)+".txt", 'r+') as feiler:
            for l in feiler:
                strip_line=l.strip()
                space_strip_line=str.split(strip_line)
                label_ind=space_strip_line[0]
                xkord=float(space_strip_line[1])
                ykord=float(space_strip_line[2])
                labelbreite=space_strip_line[3]
                labelhöhe=space_strip_line[4]

                arr=np.arange(9).reshape(3,3)
                n=int(label_ind)
                row=n//3
                column=n%3

                if mode==1:
                    if axis == 0:
                        ykordfin=1-float(ykord)
                        ykordfinal=str(ykordfin)
                        xkordfinal=xkord
                    
                    if axis==1:
                        xkordfin=1-float(xkord)
                        xkordfinal=str(xkordfin)
                        ykordfinal=ykord
                    
                    else:
                        ykordfin=1-float(ykord)
                        xkordfin=1-float(xkord)
                        xkordfinal=str(xkordfin)
                        ykordfinal=str(ykordfin)

                    if axis==0:
                        arr=np.flip(arr,axis=0)

                    if axis==1:
                        arr=np.flip(arr,axis=1)
                    
                    else:
                        arr=np.flip(arr,axis=0)
                        arr=np.flip(arr,axis=1)

                    variable=arr[row,column]
                    label_ind_fin=variable 
                    # flipped Bounding Boxes

                if mode==0:
                    arr=np.rot90(arr, -numrot)
                    variable=arr[row,column]
                    label_ind_fin=variable 

                    point=(float(xkord),float(ykord))
                    origin=(0.5,0.5)
                
                    finalmidpoint=rotate(origin,point, -1.5708*numrot)
                
                    xkordfinal=finalmidpoint[0]
                    ykordfinal=finalmidpoint[1]
                    zwischenspeicher=labelhöhe
                    zwischenspeicher2=labelbreite

                    if numrot==1 or numrot==3:
                        labelbreite=zwischenspeicher
                        labelhöhe=zwischenspeicher2
                    
                    if numrot==2:
                        labelbreite=zwischenspeicher2
                        labelhöhe=zwischenspeicher
                    # rotiert Bounding Boxes
                
                listarray=[str(label_ind_fin), str(xkordfinal), str(ykordfinal), str(labelbreite), str(labelhöhe)]
                writearray.append(listarray)

        with open (folder+"/"+str(i)+".txt", 'w+') as writer:
            for j in sorted(writearray):
                finalstring=" ".join(j)
                writer.write(finalstring+"\n")

In [13]:
# axis: nach der geflipped werden soll
    # 0 vertikal
    # 1 Horizontal (!!NUR NUMMER 1 FUNKTIONIERT GERADE WEIß NICHT WIESO ABER IST SOWIESO
    # DIE EINZIGE SPIEGELUNG DIE WIR BRAUCHEN)
    # 2 diagonal

# mode: der entscheidet ob geflipped oder rotiert wird
    # 0 rotieren
    # 1 flippen

# numrot: Anzahl an Rotierungen

augment(axis=1, mode=0, numrot=1, folder="rot90")
augment(axis=1, mode=0, numrot=2, folder="rot180")
augment(axis=1, mode=0, numrot=3, folder="rot270")
augment(axis=1, mode=1, numrot=0, folder="flipped_axis_1")
augment(axis=1, mode=1, numrot=0, folder="raw")
augment(axis=1, mode=0, numrot=1, folder="flipped_axis_1_rot90")
augment(axis=1, mode=0, numrot=2, folder="flipped_axis_1_rot180")
augment(axis=1, mode=0, numrot=3, folder="flipped_axis_1_rot270")
augment(axis=1, mode=1, numrot=0, folder="raw")

In [ ]:
os.makedirs('boardz', exist_ok=True)
n_files = 133

for i in range(n_files):
    shutil.copyfile('raw/' + str(i) + '.jpg', 'boardz/' + str(i) + '.jpg')
    shutil.copyfile('raw/' + str(i) + '.txt', 'boardz/' + str(i) + '.txt')

for i in range(n_files):
    shutil.copyfile('rot90/' + str(i) + '.jpg', 'boardz/' + str(i+n_files) + '.jpg')
    shutil.copyfile('rot90/' + str(i) + '.txt', 'boardz/' + str(i+n_files) + '.txt')

for i in range(n_files):
    shutil.copyfile('rot180/' + str(i) + '.jpg', 'boardz/' + str(i+n_files*2) + '.jpg')
    shutil.copyfile('rot180/' + str(i) + '.txt', 'boardz/' + str(i+n_files*2) + '.txt')

for i in range(n_files):
    shutil.copyfile('rot270/' + str(i) + '.jpg', 'boardz/' + str(i+n_files*3) + '.jpg')
    shutil.copyfile('rot270/' + str(i) + '.txt', 'boardz/' + str(i+n_files*3) + '.txt')

for i in range(n_files):
    shutil.copyfile('flipped_axis_1/' + str(i) + '.jpg', 'boardz/' + str(i+n_files*4) + '.jpg')
    shutil.copyfile('flipped_axis_1/' + str(i) + '.txt', 'boardz/' + str(i+n_files*4) + '.txt')

for i in range(n_files):
    shutil.copyfile('flipped_axis_1_rot90/' + str(i) + '.jpg', 'boardz/' + str(i+n_files*5) + '.jpg')
    shutil.copyfile('flipped_axis_1_rot90/' + str(i) + '.txt', 'boardz/' + str(i+n_files*5) + '.txt')

for i in range(n_files):
    shutil.copyfile('flipped_axis_1_rot180/' + str(i) + '.jpg', 'boardz/' + str(i+n_files*6) + '.jpg')
    shutil.copyfile('flipped_axis_1_rot180/' + str(i) + '.txt', 'boardz/' + str(i+n_files*6) + '.txt')

for i in range(n_files):
    shutil.copyfile('flipped_axis_1_rot270/' + str(i) + '.jpg', 'boardz/' + str(i+n_files*7) + '.jpg')
    shutil.copyfile('flipped_axis_1_rot270/' + str(i) + '.txt', 'boardz/' + str(i+n_files*7) + '.txt')

